In [96]:
pip install kafka-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [97]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

In [98]:
#1-receiving data in consumer 
try:
    producer = KafkaProducer(
        bootstrap_servers=['54.91.216.170:9092'],
        value_serializer=lambda x: dumps(x).encode('utf-8')
    )
    print("Producer created successfully")
except Exception as e:
    print("Producer error:", e)

Producer created successfully


In [99]:
#check EC2 Public IP Connected to KAFKA- if True then only it will work
print(producer.bootstrap_connected())

True


In [74]:
#1-receiving data in consumer
def on_success(record):
    print("Message delivered:", record)

def on_error(excp):
    print("ERROR:", excp)

producer.send('demo_testing_6', 
value={'hi':'kiran'}).add_callback(on_success).add_errback(on_error)

producer.flush()

Message delivered: RecordMetadata(topic='demo_testing_6', partition=0, topic_partition=TopicPartition(topic='demo_testing_6', partition=0), offset=31221, timestamp=1765274318872, checksum=None, serialized_key_size=-1, serialized_value_size=15, serialized_header_size=-1)


In [100]:
#2-read data from csv & upload to s3
df = pd.read_csv("C:\\Users\\DELL\\Documents\\DataPipeline_Projects\\Stock_Market_6\\indexProcessed.csv")

In [101]:
df.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
0,HSI,1986-12-31,2568.300049,2568.300049,2568.300049,2568.300049,2568.300049,0.0,333.879006
1,HSI,1987-01-02,2540.100098,2540.100098,2540.100098,2540.100098,2540.100098,0.0,330.213013
2,HSI,1987-01-05,2552.399902,2552.399902,2552.399902,2552.399902,2552.399902,0.0,331.811987
3,HSI,1987-01-06,2583.899902,2583.899902,2583.899902,2583.899902,2583.899902,0.0,335.906987
4,HSI,1987-01-07,2607.100098,2607.100098,2607.100098,2607.100098,2607.100098,0.0,338.923013


In [ ]:
#2-read data from csv & upload to s3
while True:
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send('demo_testing_6', value=dict_stock)
    sleep(1)

ERROR:kafka.conn:<BrokerConnection client_id=kafka-python-producer-15, node_id=0 host=54.91.216.170:9092 <connected> [IPv4 ('54.91.216.170', 9092)]>: Closing connection. KafkaConnectionError: Socket EVENT_READ without in-flight-requests
ERROR:kafka.conn:<BrokerConnection client_id=kafka-python-producer-15, node_id=0 host=54.91.216.170:9092 <connecting> [IPv4 ('54.91.216.170', 9092)]>: Connection attempt timed out
ERROR:kafka.conn:<BrokerConnection client_id=kafka-python-producer-15, node_id=0 host=54.91.216.170:9092 <connecting> [IPv4 ('54.91.216.170', 9092)]>: Closing connection. KafkaConnectionError: timeout


In [75]:
#clear data from kafka server
producer.flush()